Import Library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

2023-04-28 20:07:48.661555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Load Data Training

In [3]:
file_path = 'weather.csv'
data=pd.read_csv(file_path, delimiter=',',header=11,skipinitialspace=True)
data.head(12)

,Year,Month,Day,Hour,Minute,Temperature,Total_Precipitation,Wind_Speed,Wind_Direction
0,2018,1,15,0,0,16.40,0.0,6.73,74.48
1,2018,1,15,1,0,16.27,0.0,6.62,67.62
2,2018,1,15,2,0,16.14,0.0,6.30,59.04
3,2018,1,15,3,0,16.04,0.0,6.30,59.04
4,2018,1,15,4,0,15.97,0.0,6.19,54.46
5,2018,1,15,5,0,15.90,0.0,5.90,52.43
6,2018,1,15,6,0,15.83,0.0,5.86,47.49
7,2018,1,15,7,0,15.85,0.0,5.60,45.00
8,2018,1,15,8,0,16.24,0.0,6.62,45.00
9,2018,1,15,9,0,16.69,0.0,6.37,47.29
